In [5]:
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader
import torch.utils.data as data


In [1]:
## 验证 hn 的最后两层：倒数第二层是正向的最后一次输出，倒数第一层是反向的最后一次输出

import torch
import torch.nn as nn
 
net=nn.LSTM(3,4,3,bidirectional=True,batch_first=True) 

x=torch.rand(1,5,3)    # 序列长度为5，输入尺度为3
 
output,(hn,cn)=net(x)
print(output.shape,output)
print(hn.shape,hn)
print(hn[-2,:,:])
out = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim=1)
print(out)
# print(cn.shape,cn)

torch.Size([1, 5, 8]) tensor([[[ 0.0537, -0.0350,  0.1042, -0.0328, -0.1256,  0.3089,  0.2376,
           0.0594],
         [ 0.0811, -0.0454,  0.1583, -0.0627, -0.1226,  0.3159,  0.2177,
           0.0692],
         [ 0.0998, -0.0501,  0.1813, -0.0826, -0.1090,  0.3149,  0.1845,
           0.0602],
         [ 0.1181, -0.0558,  0.1880, -0.0942, -0.0875,  0.2982,  0.1353,
           0.0370],
         [ 0.1308, -0.0639,  0.1865, -0.1007, -0.0545,  0.2397,  0.0677,
           0.0077]]], grad_fn=<TransposeBackward0>)
torch.Size([6, 1, 4]) tensor([[[ 0.0323, -0.0563, -0.1411,  0.1248]],

        [[-0.3028, -0.0086,  0.0959,  0.4024]],

        [[-0.3394,  0.0367, -0.0657, -0.0872]],

        [[ 0.0901, -0.0560, -0.0368, -0.0956]],

        [[ 0.1308, -0.0639,  0.1865, -0.1007]],

        [[-0.1256,  0.3089,  0.2376,  0.0594]]], grad_fn=<StackBackward>)
tensor([[ 0.1308, -0.0639,  0.1865, -0.1007]], grad_fn=<SliceBackward>)
tensor([[ 0.1308, -0.0639,  0.1865, -0.1007, -0.1256,  0.3089,  0.23

In [3]:
# 验证 RNN 的隐藏层的shape，input层和中间层的不一样

# input_size, hidden_size, num_layers
rnn = nn.RNN(10, 5, 2)

inputR = torch.randn(2, 1, 10)

h0 = torch.randn(2, 1, 5)

output, hn = rnn(inputR, h0)
print(output.shape)
print(hn.shape)

print(rnn.weight_ih_l0.data.shape,rnn.weight_hh_l0.data.shape)
print(rnn.weight_ih_l1.data.shape,rnn.weight_hh_l1.data.shape)

torch.Size([2, 1, 5])
torch.Size([2, 1, 5])
torch.Size([5, 10]) torch.Size([5, 5])
torch.Size([5, 5]) torch.Size([5, 5])


In [48]:
# 验证函数 pad_sequence、pack_padded_sequence、pad_packed_sequence
train_x = [torch.Tensor([1, 1, 1, 1, 1, 1, 1]),
           torch.Tensor([2, 2, 2, 2, 2, 2]),
           torch.Tensor([3, 3, 3, 3, 3]),
           torch.Tensor([4, 4, 4, 4]),
           torch.Tensor([5, 5, 5]),
           torch.Tensor([6, 6]),
           torch.Tensor([7])]

# 1.pad_sequence 对数据进行填充，默认为0
data_length = [len(sq) for sq in train_x]
print(data_length)
x = rnn_utils.pad_sequence(train_x, batch_first=True, padding_value=0)
print(x.shape)
x=x.unsqueeze(-1)
print(x.shape)
# 2.对 0 的部分进行压缩
batch_x_pack = rnn_utils.pack_padded_sequence(x, data_length, batch_first=True)
# print( batch_x_pack)

net = nn.LSTM(1, 10, 2, batch_first=True)
out, (hn, cn) = net(batch_x_pack)

print(hn.shape)
out_pad, out_len = rnn_utils.pad_packed_sequence(out, batch_first=True)

print(out_pad.shape,out_len)

[7, 6, 5, 4, 3, 2, 1]
torch.Size([7, 7])
torch.Size([7, 7, 1])
torch.Size([2, 7, 10])
torch.Size([7, 7, 10]) tensor([7, 6, 5, 4, 3, 2, 1])


In [60]:
# tensor 与 Tensor 的区别

print(torch.FloatTensor([1,2]))
print(torch.tensor([1.0,2],requires_grad=True))


tensor([1., 2.])
tensor([1., 2.], requires_grad=True)


In [78]:
# 复现 bug
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score

def roc_auc_score_FIXED(y_true, y_pred):
#     a, _, _ = np.unique(y_true.view(y_true.dtype.descr * y_true.shape[1]), return_index=True, return_inverse=True)
    a = np.unique(y_true)
    print(a, len(a))
    if len(a) == 1: # bug in roc_auc_score
        return accuracy_score(y_true, (y_pred))
    return roc_auc_score(y_true, y_pred)


y_true = np.array([1,1,1,1])
y_scores = np.array([0.1,1,1,0])
roc_auc_score_FIXED(y_true, y_scores)

# A = [[1, 2], [3, 4], [5, 6], [1, 2]]
# A = np.array(A)   #列表类型需转为数组类型
# a, _, _ = np.unique(A.view(A.dtype.descr * A.shape[1]), return_index=True, return_inverse=True)
# print(a)     #输出为 [(1, 2) (3, 4) (5, 6)], [0 1 2], [0 1 2 0]s

[1] 1


ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [59]:
import pandas as pd
import numpy as np

# 去重
data=pd.DataFrame({'产品':['A','A','A','A'],'数量':[50,50,30,30]})
print(data)

print(data.drop_duplicates(['数量']).reset_index(drop=True))

  产品  数量
0  A  50
1  A  50
2  A  30
3  A  30
  产品  数量
0  A  50
1  A  30


In [65]:
# 英文分词
import nltk
sentence = "At eight o'clock on Thursday morning Arthur didn't feel very good."
tokens = nltk.word_tokenize(sentence)
print(tokens)

a = 89

print(a//100%10) # 百位数
print(a//10%10) # 十位数
print(a//1%10) # 个位数

['At', 'eight', "o'clock", 'on', 'Thursday', 'morning', 'Arthur', 'did', "n't", 'feel', 'very', 'good', '.']
0
8
9


In [72]:
import torch
tensor = torch.Tensor(3, 5)
print(tensor)
# torch.long() 将tensor投射为long类型
newtensor = tensor.long()
print(newtensor)
newtensor = newtensor.float()
print(newtensor)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.5835e-43, 0.0000e+00, 1.3452e-43, 0.0000e+00],
        [1.3452e-43, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.9346e-19]])
tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [83]:
a= np.array([[1],[2],[1]])
print(np.argmax(a))
print(np.argmax(a, axis=0))#0代表列
print(np.argmax(a, axis=1))#1代表行

1
[1]
[0 0 0]
